# 目的

发现j_crf在手动设定transition参数后

join_likehood 函数对于相同的输入，给出的不同输出

本nb，为了学习join_likehood 

## 结果

通过reset_parameter()

发现是含有额外参数 start_trainsition 和 end_trainsition


## 做法

初始化的适合用include_trainsition=False 关闭额外的参数

In [1]:
#export
import sys
if __name__ == '__main__': sys.path.append('..')
import exp.common as common
from pprint import pprint
def pj(*args, **kargs):
    if common.IN_JUPYTER:
        pprint(*args, **kargs)

In [2]:
#test_export
import sure

In [16]:
from exp.min_train_for_crf import inputs, LABELS, MASK, label_dic, constraints, num_label
import torch
from torch import nn
from lib.ConditionalRandomField import ConditionalRandomField as J_crf

In [90]:
j_crf = J_crf(num_label, constraints, )

In [91]:
self = j_crf
logits = inputs
mask = MASK
tags = LABELS
j_crf._joint_likelihood(logits, tags, mask)

tensor([1.4508], grad_fn=<AddBackward0>)

In [92]:
j_crf.transitions = nn.Parameter(torch.ones_like(j_crf.transitions))
j_crf._joint_likelihood(logits, tags, mask)

tensor([2.7467], grad_fn=<AddBackward0>)

In [49]:
logits, mask, tags, j_crf.transitions

(tensor([[[0.1000, 0.2000, 0.1000],
          [0.1000, 0.2000, 0.3000]]]),
 tensor([[True, True]]),
 tensor([[1, 1]]),
 Parameter containing:
 tensor([[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]], requires_grad=True))

In [31]:
# _joint_likelihood
# 即path score

batch_size, sequence_length, _ = logits.shape

# Transpose batch size and sequence dimensions:
logits = logits.transpose(0, 1)
mask = mask.transpose(0, 1).float()
tags = tags.transpose(0, 1)

# Start with the transition scores from start_tag to the first tag in each input
if self.include_start_end_transitions:
    score = self.start_transitions.index_select(0, tags[0])
else:
    score = 0.0

# Add up the scores for the observed transitions and all the inputs but the last
for i in range(sequence_length - 1):
    # Each is shape (batch_size,)
    current_tag, next_tag = tags[i], tags[i + 1]

    # The scores for transitioning from current_tag to next_tag
    transition_score = self.transitions[current_tag, next_tag]

    # The score for using current_tag
    emit_score = logits[i].gather(1, current_tag.view(batch_size, 1)).squeeze()

    # Include t 
    score = score + transition_score * mask[i + 1] + emit_score * mask[i]

# Transition from last state to "stop" state. To start with, we need to find the last tag
# for each instance.
last_tag_index = mask.sum(0).long() - 1
last_tags = tags.gather(0, last_tag_index.view(1, batch_size)).squeeze(0)

# Compute score of transitioning to `stop_tag` from each "last tag".
if self.include_start_end_transitions:
    last_transition_score = self.end_transitions.index_select(0, last_tags)
else:
    last_transition_score = 0.0

# Add the last input if it's not masked.
last_inputs = logits[-1]  # (batch_size, num_tags)
last_input_score = last_inputs.gather(1, last_tags.view(-1, 1)).squeeze()  # (batch_size)
# last_input_score = last_input_score.squeeze()  # (batch_size,)

score = score + last_transition_score + last_input_score * mask[-1]

In [32]:
score

tensor([0.5056], grad_fn=<AddBackward0>)